<a href="https://colab.research.google.com/github/usmanmalik07/Adept-Internship/blob/main/Cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from keras.datasets import cifar10

In [30]:
num_epochs =4
batch_size =4
learning_rate = 0.001

In [31]:
device = torch.device('cuda'if torch.cuda.is_available() else 'cpu')

In [32]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((
    0.5,0.5,0.5),(0.5,0.5,0.5)
)])

In [33]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                         shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [34]:
classes = ('plane','car','bird','cat','deer','frog','horse','ship','truck')

In [35]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [38]:
model = ConvNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape: [4, 3, 32, 32] = 4, 3, 1024
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 2000 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

print('Finished Training')
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)

with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

Epoch [1/4], Step [2000/12500], Loss: 2.3102
Epoch [1/4], Step [4000/12500], Loss: 2.2947
Epoch [1/4], Step [6000/12500], Loss: 2.2882
Epoch [1/4], Step [8000/12500], Loss: 2.2684
Epoch [1/4], Step [10000/12500], Loss: 2.2956
Epoch [1/4], Step [12000/12500], Loss: 2.3456
Epoch [2/4], Step [2000/12500], Loss: 1.8585
Epoch [2/4], Step [4000/12500], Loss: 1.8061
Epoch [2/4], Step [6000/12500], Loss: 2.3330
Epoch [2/4], Step [8000/12500], Loss: 1.3011
Epoch [2/4], Step [10000/12500], Loss: 1.9471
Epoch [2/4], Step [12000/12500], Loss: 1.7107
Epoch [3/4], Step [2000/12500], Loss: 1.4452
Epoch [3/4], Step [4000/12500], Loss: 1.8883
Epoch [3/4], Step [6000/12500], Loss: 1.5879
Epoch [3/4], Step [8000/12500], Loss: 1.4771
Epoch [3/4], Step [10000/12500], Loss: 1.8795
Epoch [3/4], Step [12000/12500], Loss: 1.1248
Epoch [4/4], Step [2000/12500], Loss: 1.1368
Epoch [4/4], Step [4000/12500], Loss: 1.3596
Epoch [4/4], Step [6000/12500], Loss: 1.2140
Epoch [4/4], Step [8000/12500], Loss: 1.6753
Epoc